# Xception Tuned

In [ ]:
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

Google drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip "/content/drive/MyDrive/Dataset/data.zip" -d "/content/data"
# !unzip "/content/drive/MyDrive/Dataset/data.zip"

# Loading Data generator function

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tqdm.keras import TqdmCallback
from tensorflow.keras.optimizers import Adam

In [ ]:
# # COLAB
# train_dir = 'train/'
# validation_dir = 'validation/'
# test_dir = 'test/'

# KAGGLE
train_dir = '/kaggle/input/bangla-sign/train/'
validation_dir = '/kaggle/input/bangla-sign/validation/'
test_dir = '/kaggle/input/bangla-sign/test/'

In [ ]:
# Define data generators for training, validation, and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Zoom transformations
    horizontal_flip=True  # Horizontal flips
)

# Only rescale for testing data
test_datagen = ImageDataGenerator(rescale=1./255)

# Set up data generators to read images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,  # Number of images in each batch
    class_mode='categorical'  # Since you have multiple classes
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Model

In [ ]:
# Load pre-trained Xception model (excluding top layers)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Unfreeze some top layers for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True
    
NUM_CLASSES = 38

# Add custom top layers for fine-tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)  
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout with a dropout rate of 0.5
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Combine base model with custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Create an optimizer with a lower learning rate
optimizer = Adam(learning_rate=0.0001)

# Compile the model with the custom optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Check GPU availability

In [ ]:
# Check if GPU is available
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Check TensorFlow GPU support
print("TensorFlow GPU Support:", tf.test.is_built_with_cuda())

# Train

In [ ]:
# Define the number of training and validation steps per epoch
train_steps_per_epoch = len(train_generator)
validation_steps_per_epoch = len(validation_generator)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=100,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_steps_per_epoch
)

# Evaluate the model on the test set
test_steps_per_epoch = len(test_generator)
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps_per_epoch)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)